In [1]:
import torch
import torchvision
import cv2

D:\Ott_diploma_proj_prog_files\conda\envs\ott_diploma\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [3]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = 1080
        record["width"] = 1920
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
classes = ['icy_tail', 'icy_asphalt', 'powdery_snow', 'snow_drift', 'puffy_road_snow', 'snowy_road']

data_path = 'DATA/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes, thing_colors = [
        (255,255,0),(0,0,255),(0,255,0),(255,0,255), (180,165,0), (187,132,156)]) #Collored by me
  
    

microcontroller_metadata = MetadataCatalog.get("category_train")

In [5]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 25000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6

In [7]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[03/10 17:40:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[03/10 17:40:18 d2.data.build]: Removed 0 images with no usable annotations. 152 images left.
[03/10 17:40:18 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |   category    | #instances   |   category   | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:------------:|:-------------|
|  icy_tail  | 62           |  icy_asphalt  | 82           | powdery_snow | 30           |
| snow_drift | 313          | puffy_road_.. | 189          |  snowy_road  | 232          |
|            |              |               |              |              |              |
|   total    | 908          |               |              |              |              |
[03/10 17:40:18 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[03/10 17:40:18 d2.data.build]: Using training sampler TrainingSam

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

In [8]:
trainer.train()

[03/10 17:40:38 d2.engine.train_loop]: Starting training from iteration 0


C:\Users\gvo2\AppData\Roaming\Python\Python37\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/10 17:41:04 d2.utils.events]:  eta: 5:42:58  iter: 19  total_loss: 3.85  loss_cls: 1.875  loss_box_reg: 0.1091  loss_mask: 0.6924  loss_rpn_cls: 0.9838  loss_rpn_loc: 0.1988  time: 0.8872  last_time: 0.7866  data_time: 0.1038  last_data_time: 0.0031   lr: 4.9953e-06  max_mem: 4813M
[03/10 17:41:21 d2.utils.events]:  eta: 5:37:53  iter: 39  total_loss: 3.38  loss_cls: 1.699  loss_box_reg: 0.1463  loss_mask: 0.6929  loss_rpn_cls: 0.6577  loss_rpn_loc: 0.1837  time: 0.8651  last_time: 0.7991  data_time: 0.0030  last_data_time: 0.0027   lr: 9.9902e-06  max_mem: 4813M
[03/10 17:41:38 d2.utils.events]:  eta: 5:44:26  iter: 59  total_loss: 2.862  loss_cls: 1.447  loss_box_reg: 0.1904  loss_mask: 0.6915  loss_rpn_cls: 0.3971  loss_rpn_loc: 0.1488  time: 0.8573  last_time: 0.8499  data_time: 0.0028  last_data_time: 0.0028   lr: 1.4985e-05  max_mem: 4813M
[03/10 17:41:55 d2.utils.events]:  eta: 5:43:10  iter: 79  total_loss: 2.505  loss_cls: 1.047  loss_box_reg: 0.1811  loss_mask: 0.6894  lo

[03/10 17:50:10 d2.utils.events]:  eta: 5:59:43  iter: 599  total_loss: 1.383  loss_cls: 0.3472  loss_box_reg: 0.3514  loss_mask: 0.4619  loss_rpn_cls: 0.04172  loss_rpn_loc: 0.1284  time: 0.9285  last_time: 0.8721  data_time: 0.0028  last_data_time: 0.0027   lr: 0.00014985  max_mem: 4813M
[03/10 17:50:40 d2.utils.events]:  eta: 5:59:45  iter: 619  total_loss: 1.218  loss_cls: 0.2984  loss_box_reg: 0.3175  loss_mask: 0.461  loss_rpn_cls: 0.04321  loss_rpn_loc: 0.1318  time: 0.9478  last_time: 0.9166  data_time: 0.0028  last_data_time: 0.0033   lr: 0.00015485  max_mem: 4813M
[03/10 17:51:02 d2.utils.events]:  eta: 5:59:59  iter: 639  total_loss: 1.32  loss_cls: 0.3575  loss_box_reg: 0.3726  loss_mask: 0.4407  loss_rpn_cls: 0.04135  loss_rpn_loc: 0.148  time: 0.9523  last_time: 1.2130  data_time: 0.0028  last_data_time: 0.0025   lr: 0.00015984  max_mem: 4813M
[03/10 17:51:21 d2.utils.events]:  eta: 5:59:46  iter: 659  total_loss: 1.28  loss_cls: 0.2977  loss_box_reg: 0.3201  loss_mask: 0

[03/10 18:00:11 d2.utils.events]:  eta: 5:59:13  iter: 1179  total_loss: 0.9467  loss_cls: 0.2379  loss_box_reg: 0.2565  loss_mask: 0.3053  loss_rpn_cls: 0.02053  loss_rpn_loc: 0.1249  time: 0.9786  last_time: 0.9123  data_time: 0.0028  last_data_time: 0.0032   lr: 0.00025  max_mem: 4813M
[03/10 18:00:30 d2.utils.events]:  eta: 5:59:05  iter: 1199  total_loss: 1.006  loss_cls: 0.2487  loss_box_reg: 0.255  loss_mask: 0.2942  loss_rpn_cls: 0.02518  loss_rpn_loc: 0.1325  time: 0.9777  last_time: 0.9393  data_time: 0.0029  last_data_time: 0.0035   lr: 0.00025  max_mem: 4813M
[03/10 18:00:50 d2.utils.events]:  eta: 5:59:10  iter: 1219  total_loss: 1.035  loss_cls: 0.2747  loss_box_reg: 0.2999  loss_mask: 0.3278  loss_rpn_cls: 0.02205  loss_rpn_loc: 0.1196  time: 0.9780  last_time: 0.9091  data_time: 0.0029  last_data_time: 0.0034   lr: 0.00025  max_mem: 4813M
[03/10 18:01:10 d2.utils.events]:  eta: 5:59:23  iter: 1239  total_loss: 1.063  loss_cls: 0.2874  loss_box_reg: 0.3029  loss_mask: 0.

...

[03/10 23:33:51 d2.utils.events]:  eta: 0:36:01  iter: 22679  total_loss: 0.2057  loss_cls: 0.0278  loss_box_reg: 0.0598  loss_mask: 0.07094  loss_rpn_cls: 0.0006936  loss_rpn_loc: 0.04589  time: 0.9268  last_time: 0.9540  data_time: 0.0029  last_data_time: 0.0029   lr: 0.00025  max_mem: 5029M
[03/10 23:34:09 d2.utils.events]:  eta: 0:35:42  iter: 22699  total_loss: 0.2142  loss_cls: 0.03081  loss_box_reg: 0.06198  loss_mask: 0.07563  loss_rpn_cls: 0.0007993  loss_rpn_loc: 0.04544  time: 0.9268  last_time: 0.9801  data_time: 0.0029  last_data_time: 0.0035   lr: 0.00025  max_mem: 5029M
[03/10 23:34:28 d2.utils.events]:  eta: 0:35:23  iter: 22719  total_loss: 0.2027  loss_cls: 0.02816  loss_box_reg: 0.06175  loss_mask: 0.07143  loss_rpn_cls: 0.0003788  loss_rpn_loc: 0.04302  time: 0.9268  last_time: 0.9239  data_time: 0.0030  last_data_time: 0.0026   lr: 0.00025  max_mem: 5029M
[03/10 23:34:47 d2.utils.events]:  eta: 0:35:04  iter: 22739  total_loss: 0.2134  loss_cls: 0.02814  loss_box_r

[03/10 23:42:34 d2.utils.events]:  eta: 0:27:18  iter: 23239  total_loss: 0.2035  loss_cls: 0.02787  loss_box_reg: 0.06227  loss_mask: 0.07079  loss_rpn_cls: 0.0006484  loss_rpn_loc: 0.04141  time: 0.9268  last_time: 0.9326  data_time: 0.0029  last_data_time: 0.0035   lr: 0.00025  max_mem: 5029M
[03/10 23:42:53 d2.utils.events]:  eta: 0:26:59  iter: 23259  total_loss: 0.2148  loss_cls: 0.0255  loss_box_reg: 0.06003  loss_mask: 0.07241  loss_rpn_cls: 0.0006313  loss_rpn_loc: 0.04501  time: 0.9268  last_time: 0.9481  data_time: 0.0029  last_data_time: 0.0028   lr: 0.00025  max_mem: 5029M
[03/10 23:43:11 d2.utils.events]:  eta: 0:26:41  iter: 23279  total_loss: 0.1981  loss_cls: 0.02807  loss_box_reg: 0.06031  loss_mask: 0.07441  loss_rpn_cls: 0.0007659  loss_rpn_loc: 0.04355  time: 0.9268  last_time: 0.9573  data_time: 0.0029  last_data_time: 0.0033   lr: 0.00025  max_mem: 5029M
[03/10 23:43:30 d2.utils.events]:  eta: 0:26:23  iter: 23299  total_loss: 0.1999  loss_cls: 0.02926  loss_box_

[03/10 23:51:16 d2.utils.events]:  eta: 0:18:35  iter: 23799  total_loss: 0.1923  loss_cls: 0.02654  loss_box_reg: 0.06035  loss_mask: 0.06957  loss_rpn_cls: 0.0006409  loss_rpn_loc: 0.0394  time: 0.9267  last_time: 0.9225  data_time: 0.0029  last_data_time: 0.0028   lr: 0.00025  max_mem: 5029M
[03/10 23:51:34 d2.utils.events]:  eta: 0:18:17  iter: 23819  total_loss: 0.2127  loss_cls: 0.02906  loss_box_reg: 0.06619  loss_mask: 0.07242  loss_rpn_cls: 0.0005945  loss_rpn_loc: 0.04708  time: 0.9268  last_time: 0.9793  data_time: 0.0030  last_data_time: 0.0027   lr: 0.00025  max_mem: 5029M
[03/10 23:51:53 d2.utils.events]:  eta: 0:17:58  iter: 23839  total_loss: 0.2169  loss_cls: 0.0302  loss_box_reg: 0.0689  loss_mask: 0.07126  loss_rpn_cls: 0.0007342  loss_rpn_loc: 0.04436  time: 0.9268  last_time: 0.9547  data_time: 0.0029  last_data_time: 0.0030   lr: 0.00025  max_mem: 5029M
[03/10 23:52:12 d2.utils.events]:  eta: 0:17:40  iter: 23859  total_loss: 0.2101  loss_cls: 0.03078  loss_box_re

[03/10 23:59:58 d2.utils.events]:  eta: 0:09:55  iter: 24359  total_loss: 0.2036  loss_cls: 0.02835  loss_box_reg: 0.06155  loss_mask: 0.07098  loss_rpn_cls: 0.0008928  loss_rpn_loc: 0.04571  time: 0.9268  last_time: 0.9581  data_time: 0.0031  last_data_time: 0.0036   lr: 0.00025  max_mem: 5029M
[03/11 00:00:17 d2.utils.events]:  eta: 0:09:37  iter: 24379  total_loss: 0.1973  loss_cls: 0.02688  loss_box_reg: 0.06031  loss_mask: 0.06738  loss_rpn_cls: 0.0005873  loss_rpn_loc: 0.04629  time: 0.9268  last_time: 0.9163  data_time: 0.0029  last_data_time: 0.0028   lr: 0.00025  max_mem: 5029M
[03/11 00:00:36 d2.utils.events]:  eta: 0:09:18  iter: 24399  total_loss: 0.2034  loss_cls: 0.02777  loss_box_reg: 0.05808  loss_mask: 0.06613  loss_rpn_cls: 0.0005251  loss_rpn_loc: 0.04241  time: 0.9268  last_time: 0.9658  data_time: 0.0029  last_data_time: 0.0028   lr: 0.00025  max_mem: 5029M
[03/11 00:00:54 d2.utils.events]:  eta: 0:09:00  iter: 24419  total_loss: 0.199  loss_cls: 0.02568  loss_box_

[03/11 00:08:41 d2.utils.events]:  eta: 0:01:14  iter: 24919  total_loss: 0.2057  loss_cls: 0.02605  loss_box_reg: 0.05832  loss_mask: 0.07189  loss_rpn_cls: 0.0006245  loss_rpn_loc: 0.04461  time: 0.9268  last_time: 0.9936  data_time: 0.0031  last_data_time: 0.0033   lr: 0.00025  max_mem: 5029M
[03/11 00:09:00 d2.utils.events]:  eta: 0:00:55  iter: 24939  total_loss: 0.1864  loss_cls: 0.02856  loss_box_reg: 0.05324  loss_mask: 0.06793  loss_rpn_cls: 0.0006252  loss_rpn_loc: 0.04368  time: 0.9268  last_time: 0.8616  data_time: 0.0028  last_data_time: 0.0027   lr: 0.00025  max_mem: 5029M
[03/11 00:09:19 d2.utils.events]:  eta: 0:00:37  iter: 24959  total_loss: 0.2141  loss_cls: 0.02872  loss_box_reg: 0.05831  loss_mask: 0.0695  loss_rpn_cls: 0.000902  loss_rpn_loc: 0.04634  time: 0.9268  last_time: 0.9638  data_time: 0.0029  last_data_time: 0.0028   lr: 0.00025  max_mem: 5029M
[03/11 00:09:37 d2.utils.events]:  eta: 0:00:18  iter: 24979  total_loss: 0.1974  loss_cls: 0.02426  loss_box_r

In [9]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4 
cfg.DATASETS.TEST = ("test", )
predictor = DefaultPredictor(cfg)

[03/11 00:09:59 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output\model_final.pth ...


In [10]:
class MyVisualizer(Visualizer):
    def _jitter(self, color ):
        return color

In [11]:
img = cv2.imread(data_path+'test/165.jpg')
outputs = predictor(img)
v = MyVisualizer(img[:, :, ::-1],
                metadata=microcontroller_metadata, 
                scale=0.8, 
                instance_mode=ColorMode.SEGMENTATION, 
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (16, 9))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

AttributeError: 'NoneType' object has no attribute 'shape'

In [12]:
from detectron2.modeling import build_model

In [13]:
my_model = build_model(cfg)

In [14]:
my_model

GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): BottleneckBlock